## Import necessary libraries

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os
import threading
import sys
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout, QPushButton, QHBoxLayout,QStackedLayout


C:\Users\New_User\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load the weights

In [2]:
emotion_model = Sequential() #layer by layer
emotion_model.add(Conv2D(32 , kernel_size = (3,3), activation = 'relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64 , kernel_size = (3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128 , kernel_size = (3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Conv2D(128 , kernel_size = (3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.25))
emotion_model.add(Dense(7, activation='softmax'))
#loading the weights
emotion_model.load_weights('model.weights.h5')
#Disables the use of OpenCL in OpenCV
cv2.ocl.setUseOpenCL(False)

C:\Users\New_User\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Initialization

In [3]:
last_frame1 = np.zeros((480, 640, 3), dtype=np.uint8)
cap1 = None
show_text = [0]
frame_number = 0  # Initialize frame_number
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}  # Example
emojis_dir = "emojis"

# Dictionary mapping emotions to emoji file paths
emoji_dist = {
    0: os.path.join(emojis_dir, "angry.jpg"),
    1: os.path.join(emojis_dir, "disgust.jpeg"),
    2: os.path.join(emojis_dir, "fear.png"),
    3: os.path.join(emojis_dir, "happy.jpg"),
    4: os.path.join(emojis_dir, "neutral.png"),
    5: os.path.join(emojis_dir, "sad.jpeg"),
    6: os.path.join(emojis_dir, "surprised.png")
}


In [4]:
class PhotoApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    
    def initUI(self):
        self.setWindowTitle("Photo en Emoji")
        self.setGeometry(100, 100, 1000, 700)
        self.setStyleSheet("background-color: white;")

        # Create a horizontal layout to hold the camera feed and emoji display
        h_layout = QHBoxLayout()
        h_layout.setSpacing(20)  # Add spacing between widgets

        self.label = QLabel(self)
        self.label.setFixedSize(600, 500)  # Set fixed size for camera feed
        self.label.setStyleSheet("border: 2px solid #000;")  # Add border for clarity
        h_layout.addWidget(self.label, alignment=Qt.AlignCenter)

        # Create a stacked layout for the emoji and emotion text
        stacked_layout = QStackedLayout()

        self.label2 = QLabel(self)
        self.label2.setFixedSize(400, 400)  # Set fixed size for emoji display
        self.label2.setStyleSheet("border: 2px solid #000;")  # Add border for clarity
        stacked_layout.addWidget(self.label2)

        self.label3 = QLabel(self)
        self.label3.setFixedSize(400, 50)
        self.label3.setStyleSheet("color: #555555; font-size: 24px; background: transparent;")
        self.label3.setAlignment(Qt.AlignCenter | Qt.AlignBottom)  # Align text at the bottom center
        stacked_layout.addWidget(self.label3)

        stacked_widget = QWidget()
        stacked_widget.setLayout(stacked_layout)
        h_layout.addWidget(stacked_widget, alignment=Qt.AlignCenter)

        # Create a vertical layout to hold the horizontal layout and the "Quit" button
        v_layout = QVBoxLayout()
        v_layout.setContentsMargins(20, 20, 20, 20)  # Set margins for the main layout
        v_layout.addLayout(h_layout)

        # Add stretch to push the button to the bottom
        v_layout.addStretch()

        # Create and add the "Quit" button
        self.exitButton = QPushButton("Quit", self)
        self.exitButton.setStyleSheet("font: bold 25px; color: red; padding: 10px 20px;")
        self.exitButton.clicked.connect(self.close)
        v_layout.addWidget(self.exitButton, alignment=Qt.AlignRight)

        # Set the main layout of the widget
        self.setLayout(v_layout)

        # Setup QTimer for video capture
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(10)

        # Setup QTimer for updating emoji
        self.emoji_timer = QTimer()
        self.emoji_timer.timeout.connect(self.update_emoji)
        self.emoji_timer.start(10)

        self.show()

    #captures frames from the camera, performs face detection, 
    #predicts emotions for detected faces, 
    #displays the processed frames with annotations using PyQt5 widgets.
    def update_frame(self):
        global cap1, frame_number, last_frame1
        if cap1 is None:
            cap1 = cv2.VideoCapture(0)
            if not cap1.isOpened():
                print("Unable to open the camera!")
                return

        flag1, frame1 = cap1.read()
        if not flag1:
            print("Failed to read frame")
            return

        if frame1 is None:
            print("Empty frame received!")
            return

        frame1 = cv2.resize(frame1, (600, 500))
        bound_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        n_faces = bound_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        
        for (x, y, w, h) in n_faces:
            #For each detected face, a rectangle is drawn around it
            cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
            roi_frame = gray_frame[y:y + h, x:x + w]
            crop_img = np.expand_dims(np.expand_dims(cv2.resize(roi_frame, (48, 48)), -1), 0)
            prediction = emotion_model.predict(crop_img)
            maxindex = int(np.argmax(prediction))
            cv2.putText(frame1, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            show_text[0] = maxindex

        last_frame1 = frame1.copy()
        img = cv2.cvtColor(last_frame1, cv2.COLOR_BGR2RGB)
        img = QImage(img.data, img.shape[1], img.shape[0], QImage.Format_RGB888)
        pix = QPixmap.fromImage(img)
        self.label.setPixmap(pix)

    def update_emoji(self):
        frame2 = cv2.imread(emoji_dist[show_text[0]])
        if frame2 is None:
            print(f"Failed to load emoji image: {emoji_dist[show_text[0]]}")
            return

        frame2 = cv2.resize(frame2, (400, 400))  # Resize emoji for display
        img2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        img2 = QImage(img2.data, img2.shape[1], img2.shape[0], QImage.Format_RGB888)
        pix2 = QPixmap.fromImage(img2)
        self.label2.setPixmap(pix2)
        self.label3.setText(emotion_dict[show_text[0]])
        
    def closeEvent(self, event):
        if cap1 is not None:
            cap1.release()
        event.accept()



In [5]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = PhotoApp()
    sys.exit(app.exec_())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━

SystemExit: 0

C:\Users\New_User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
